In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import re
import time
import dateutil.parser
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

In [107]:
# Create all functions required for web scraping
def find_title_links(soup):
    '''Given a beautiful soup object with movie title links, 
    get movie titles from link tags from BOM genre and actor/actress pages'''
    titles = []
    for item in soup.find_all('a'):
        children = []
        if "/movies/?id" in str(item):
            for child in item.descendants:
                children.append(child)
            for child in children:
                if "<b>" not in str(child.encode('utf-8')) and '<font color="#666666">' not in str(child.encode('utf-8')):
                    titles.append(str(child.encode('utf-8')))
    return titles

def get_movie_titles(links):
    '''Given a list of links for BOM genre and actor/actress pages,
    create a beautiful soup object, pull titles for each link,
    and return a list of unique movie titles'''
    movie_titles = []
    for link in links:
        url = link
        response = requests.get(url)
        #assert response.status_code == 200
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        table_movies = soup.find_all('table')[2]
        titles = find_title_links(table_movies)
        for title in titles:
            movie_titles.append(title)
    return list(set(movie_titles))

def split_titles_yr_noyr(titles):
    '''Given a list of titles pulled from BOM genre and actor/actress pages,
    split them into two lists -- one without a year specified
    and one with a list of tuples containing title and year -- and return list of lists 
    (to ensure the API query doesn't break)'''
    titles_noyr = []
    titles_yr = []
    year = re.compile(r"\s\([0-9]{4,4}\)") #regex identifies presence of year in parentheses
    parens = re.compile(r"\s\(.+\)(\s)?") #regex identifies presence of other cases of parenttheses
    for item in titles:
        sy = year.search(item)
        sp = parens.search(item)
        if sy:
            titles_yr.append((item[:sy.start()], item[(sy.start()+2):(sy.start()+6)])) #split title and year into tuple
        elif sp:
            titles_noyr.append(item[:sp.start()])
        else:
            titles_noyr.append(item)
    return [titles_noyr, titles_yr]

def get_api_query_title(titles):
    '''Given a list of titles with and without year specified, build links to query the OMDB API
    for cases with and without year present, and return a list of links'''
    query = []
    for title in titles[0]:
        query.append("http://www.omdbapi.com/?t={}&type=movie&plot=short&tomatoes=true".format(title))
    for title_year in titles[1]:
        query.append("http://www.omdbapi.com/?t={}&y={}&type=movie&plot=short&tomatoes=true".format(title_year[0], title_year[1]))
    return query

def get_api_movie_data_title(query):
    '''Given a list of links for movie titles in query format, query OMDB API urls, 
    scrape movie data, and return a dataframe with results by title'''
    movie_data = []
    for link in query:  
        url = link
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        paragraph = soup.find('p')
        s_dict = paragraph.contents[0]
        title_dict = json.loads(s_dict)
        movie_data.append(title_dict)
    return pd.DataFrame(movie_data)

def get_bom_actor_links(actors):
    '''Given a list of actor/actress names, format with BOM link structure for getting
    to actor/actress page of movie titles, and return a list of links'''
    formatted = []
    links = []
    for actor in actors:
        nospace = actor.replace(" ", "")
        low = nospace.lower()
        formatted.append(low)
    for actor in formatted:
        links.append("http://www.boxofficemojo.com/people/chart/?view=Actor&id={}.htm".format(actor))
    return links

def get_title_actor_df(links):
    """Given a list of actor/actress BOM links, return a dataframe that acts as a crosswalk
    of actor/actress name and movie title"""
    movie_dicts = []
    headers = ['Actors', 'Title']
    for link in links:
        reg = re.compile(r"id=(.+).htm")
        actor = re.findall(reg, link)[0]
        url = link
        response = requests.get(url)
        #assert response.status_code == 200
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        table_movies = soup.find_all('table')[2]
        titles = find_title_links(table_movies)
        for title in titles:
            movie_actor_dict = dict(zip(headers, [actor, title]))
            movie_dicts.append(movie_actor_dict)
    return pd.DataFrame(movie_dicts)

def get_title_bomlinks_by_actor(links):
    '''Given a list of actor/actress BOM links, find all titles for that actor/actress
    and return a unique list of movie title links formatted to scrape movie data from BOM'''
    movie_links = []
    for link in links:
        link_stem = []
        url = link
        response = requests.get(url)
        #assert response.status_code == 200
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        table_movies = soup.find_all('table')[2]
        for link in table_movies.find_all('a'):
            if "/movies/?id" in str(link):
                if link.has_attr('href'):
                    link_stem.append(link['href'])
        for link in link_stem:
            movie_links.append("http://www.boxofficemojo.com{}".format(link.encode('utf-8')))
    return list(set(movie_links))

def get_title_bomlinks_by_genre(links):
    '''Given a list of genre BOM links, find all titles for that genre
    and return a unique list of movie title links formatted to scrape movie data from BOM'''
    movie_links = []
    for link in links:
        link_stem = []
        url = link
        response = requests.get(url)
        #assert response.status_code == 200
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        table_movies = soup.find_all('table')[2]
        for link in table_movies.find_all('a'):
            if "/movies/?id" in str(link):
                if link.has_attr('href'):
                    link_stem.append(link['href'])
        for link in link_stem:
            movie_links.append("http://www.boxofficemojo.com{}".format(link.encode('utf-8')))
    return list(set(movie_links))

def get_bom_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return None

def dtg_to_int(moneystring):
    '''Takes a string money value with a dollar sign and commas and returns an integer'''
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    except:
        return None

def runtime_to_minutes(runtimestring):
    '''Takes a string for running time of a movie and returns time in minutes'''
    try:
        runtime = runtimestring.split()
        try:
            minutes = int(runtime[0])*60 + int(runtime[2])
            return minutes
        except:
            return None
    except:
        return None

def budget_to_int(moneystring):
    '''Takes a string money value for a production budget and returns an integer'''
    try:
        moneystring = moneystring.replace('$', '').replace(' million', '000000')
        return moneystring
    except:
        return None

def get_theaters(soup):
    '''Gets theater release number from BOM movie page'''
    try:
        raw_theaters = soup.find_all(class_='mp_box_content')[1].find_all('td')[7]
        return raw_theaters
    except:
        return None

def theaters_to_int(raw_theaters):
    '''Takes a theater release number string and returns an integer'''
    try:
        theaters = int(raw_theaters.text.split()[0].replace(",", ''))
        return theaters
    except:
        return None

def get_inrelease(soup):
    '''Gets string with days/weeks in release from BOM movie page'''
    try:
        raw_inrelease = soup.find_all(class_='mp_box_content')[1].find_all('td')[11]
        return raw_inrelease
    except:
        return None
    
def get_inrelease_days(inrelease):
    '''Takes string with days/weeks in release and returns days as integer'''
    try:
        return int(inrelease.text.split()[0])
    except:
        return None

def get_inrelease_weeks(inrelease):
    '''Takes string with days/weeks in release and returns weeks as integer'''
    try:
        return int(inrelease.text.split()[3])
    except:
        return None

def to_date(datestring):
    '''Takes a date string from BOM release date from movie page and returns datetime object'''
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return None
    
def to_year(datetime_obj):
    '''Takes datetime object for BOM release date and returns the year'''
    try:
        year = datetime_obj.year
        return year
    except:
        return None

def get_bom_title_data(links):
    '''Scrapes BOM movie pages and returns a dataframe for key variables if available'''
    bom_title_data = []
    headers = ['Title', 'DomesticTotalGross',
               'RuntimeMins', 'Budget', 'Theaters', 'ReleaseDays', 'ReleaseWeeks', 'ReleaseDate',
              'ReleaseYear'
              ]
    for link in links:
        url = link
        response = requests.get(url)
        #assert response.status_code == 200
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        raw_title = soup.find('title').text
        title = raw_title.split('(')[0].strip()
        raw_domestic_total_gross = get_bom_movie_value(soup,'Domestic Total')
        domestic_total_gross = dtg_to_int(raw_domestic_total_gross)
        raw_runtime = get_bom_movie_value(soup,'Runtime')
        runtime = runtime_to_minutes(raw_runtime)
        raw_budget = get_bom_movie_value(soup, 'Production Budget: ')
        budget = budget_to_int(raw_budget)
        raw_theaters = get_theaters(soup)
        theaters = theaters_to_int(raw_theaters)
        raw_inrelease = get_inrelease(soup)
        inrelease_days = get_inrelease_days(raw_inrelease)
        inrelease_weeks = get_inrelease_weeks(raw_inrelease)
        raw_release_date = get_bom_movie_value(soup,'Release Date')
        release_date = to_date(raw_release_date)
        release_year = to_year(release_date)
        movie_dict = dict(zip(headers, [title,
                                        domestic_total_gross,
                                        runtime,
                                        budget,
                                        theaters,
                                        inrelease_days,
                                        inrelease_weeks,
                                        release_date,
                                        release_year
                                       ]))
        bom_title_data.append(movie_dict)
        #time.sleep(5)
    return pd.DataFrame(bom_title_data)

""" split_apply_combine
in:    df_to_split - dataframe, sorted ascending by dates
       col_to_split - column containing sorted ascending dates
       col_id - column [director / actor / writer]
out:   combined - df with new avg. stat
"""
def split_apply_combine(df_to_split, col_to_split, col_id):
    split = pd.merge(df_to_split, df_to_split, on=col_id)
    apply_ = split[split[col_to_split+'_x'] >= split[col_to_split+'_y']] 
    combined = apply_.groupby([col_id,col_to_split+'_x'], as_index=False).mean()
    combined.rename(columns={col_to_split+'_x' : col_to_split}, inplace=True)
    return combined

""" split_apply_combine
in:    sorted_df - dataframe, grouped by col_to_match and sorted ascending by dates
       col_to_match - column df is grouped by
       col_get_prev - value to get prev item of, if group matches, else return mean of prev
       new_col - name of new column
out:   combined - df with new avg. stat
"""
def get_last_on_match(sorted_df, col_to_match, col_get_prev, new_col):
    
    sorted_df[new_col] = sorted_df[col_get_prev]
    
    last = sorted_df.irow(0)
    for i in range(1, sorted_df.shape[0]):
        if (last[col_to_match] == sorted_df[col_to_match].irow(i)):
            sorted_df[new_col][i] = last[col_get_prev]
        else:
            #if first movie, set it to global mean (***for now set to -1)
            #sorted_df[new_col][i] = sorted_df[col_get_prev].mean()
            sorted_df[new_col][i] = np.float64(-1.00)
        last = sorted_df.irow(i) 
        
    return(sorted_df)

""" add_rolling_average
in:    df - dataframe, grouped by col_to_match and sorted ascending by dates
       date_col - column of dates
       col_to_avg - column to average by dates
       col_to_groupby - column to groupby for rolling mean
       new_colname - name of new column to add
       how=inner - type of merge to do (default: inner)
out:   final - merged DF
"""
def add_rolling_average(df, date_col, col_to_avg, col_to_groupby, new_colname, how='inner'):
    newdf = df[[date_col, col_to_avg, col_to_groupby]]
    combined = split_apply_combine(newdf, date_col, col_to_groupby)
    final = pd.merge(combined, df, on=[col_to_groupby, date_col], how=how)
    df2 = final.sort([col_to_groupby, date_col], ascending=[1,1])
    final2 = get_last_on_match(df2, col_to_groupby, col_to_avg+'_y', new_colname)
    trim = final2.drop([col_to_avg+'_x',col_to_avg+'_y'], axis=1)
    return(trim)

# Step 1
Pull a list of movie titles from Box Office Mojo for the comedy genre, get a dataframe with movie data about each title using OMDB API, and save scraped data

In [15]:
# Create a list of links to all categories of BOM comedy genre to scrape movie titles from
movies_comedy = [
"http://www.boxofficemojo.com/genres/chart/?id=arresteddevelopment.htm",
"http://www.boxofficemojo.com/genres/chart/?id=bodyswitchcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=bumblingcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=collegecomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=blackcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=fatsuit.htm",
"http://www.boxofficemojo.com/genres/chart/?id=daddyfishoutofwater.htm",
"http://www.boxofficemojo.com/genres/chart/?id=godcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=highschoolcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=musiccomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=r-ratedcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=roadtrip.htm",
"http://www.boxofficemojo.com/genres/chart/?id=comedysequel.htm",
"http://www.boxofficemojo.com/genres/chart/?id=spoof.htm",
"http://www.boxofficemojo.com/genres/chart/?id=spycomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=whatifcomedy.htm",
"http://www.boxofficemojo.com/genres/chart/?id=comedyremake.htm"
]

In [17]:
# Get list of movie titles for comedy genre and split into year vs. no year list
titles = split_titles_yr_noyr(get_movie_titles(movies_comedy))
print len(titles[0]) + len(titles[1])
print titles[0][:5]
print titles[1][:5]

690
['Ernest Goes to Jail', 'Vampires Suck', 'Superhero Movie', 'Serving Sara', 'Senseless']
[('Heaven Can Wait', '1978'), ('Nine Lives', '2016'), ('Revenge of the Nerds', '2007'), ('Unfaithfully Yours', '1984'), ('The Nutty Professor', '1996')]


In [18]:
# Get dataframe with OMBD API movie data
movie_omdb = get_api_movie_data_title(get_api_query_title(titles))

In [19]:
movie_omdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690 entries, 0 to 689
Data columns (total 36 columns):
Actors               662 non-null object
Awards               662 non-null object
BoxOffice            662 non-null object
Country              662 non-null object
DVD                  662 non-null object
Director             662 non-null object
Error                28 non-null object
Genre                662 non-null object
Language             662 non-null object
Metascore            662 non-null object
Plot                 662 non-null object
Poster               662 non-null object
Production           662 non-null object
Rated                662 non-null object
Released             662 non-null object
Response             690 non-null object
Runtime              662 non-null object
Title                662 non-null object
Type                 662 non-null object
Website              662 non-null object
Writer               662 non-null object
Year                 662 non-null o

In [20]:
movie_omdb.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,tomatoFresh,tomatoImage,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoURL,tomatoUserMeter,tomatoUserRating,tomatoUserReviews
0,"Jim Varney, Gailard Sartain, Bill Byrge, Barba...",N/A,N/A,USA,03 Sep 2002,John R. Cherry III,NaN,"Comedy, Crime, Family",English,N/A,...,1,rotten,13,3.8,8,7,http://www.rottentomatoes.com/m/ernest_goes_to...,47,2.6,42663
1,"Jenn Proske, Matt Lanter, Diedrich Bader, Chri...",6 nominations.,$36.7M,USA,30 Nov 2010,"Jason Friedberg, Aaron Seltzer",NaN,Comedy,"English, German",18,...,4,rotten,5,2.2,88,84,http://www.rottentomatoes.com/m/vampires_suck/,29,2.5,75060
2,"Drake Bell, Sara Paxton, Christopher McDonald,...",1 win.,$25.8M,USA,08 Jul 2008,Craig Mazin,NaN,"Action, Comedy, Sci-Fi",English,33,...,8,rotten,16,3.6,51,43,http://www.rottentomatoes.com/m/1159618-superh...,32,2.8,156796
3,"Matthew Perry, Elizabeth Hurley, Vincent Pasto...",N/A,$16.9M,"Germany, USA",28 Jan 2003,Reginald Hudlin,NaN,"Comedy, Romance","English, Spanish",18,...,5,rotten,4,3.0,112,107,http://www.rottentomatoes.com/m/serving_sara/,26,2.4,27050
4,"Marlon Wayans, Brad Dourif, Esther Scott, Debr...",N/A,N/A,USA,07 Dec 1999,Penelope Spheeris,NaN,"Comedy, Romance",English,36,...,1,rotten,6,3.4,16,15,http://www.rottentomatoes.com/m/1081501-sensel...,46,2.9,10047


In [22]:
#Clean out Error records (movie not found in the API), drop Error column, and save data as csv
movie_omdb = movie_omdb[movie_omdb['Error'] != 'Movie not found!']
movie_omdb.drop('Error', inplace = True, axis = 1)
movie_omdb.to_csv("movies_comedy_omdb.csv", encoding='utf-8', index = False)

# Step 2
Restrict the list of comedy movies to only films released from the year 2000 to present, identify the list of actors/actresses appearing in those films, pull movie data for full film history for each actor/actress, and create cumulative average scores for each actor-film.

In [3]:
movie_omdb = pd.read_csv("movies_comedy_omdb.csv")

In [ ]:
movie_omdb_2000 = movie_omdb[
    (movie_omdb['Year'] >= 2000) & (pd.notnull(movie_omdb['Actors']))]

In [ ]:
# Create a series that splits cells with multiple actors/actresses into 
# individual rows, strip white space from around the names, and give the 
# series the name Actors
s = movie_omdb_2000['Actors'].str.split(',').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'Actor' # needs a name to join
s_cl = s.str.strip()
print s_cl[0:5]

In [ ]:
# Create a copy of the unique by movie dataframe, drop the Actors field,
# and join the series on index to create a movie-actor level dataframe
movie_omdb_actors_2000 = movie_omdb_2000.copy()
movie_omdb_actors_2000 = movie_omdb_actors_2000.join(s_cl)

In [ ]:
# Create a new dataframe with each actor/actress and the count of films
actors_to_get = movie_omdb_actors_2000[['Actor', 'Title']].groupby(
    'Actor', as_index = False).count()
actors_to_get.columns = ['Actor', 'TitleCnt']

In [ ]:
# Create a lowercase concatenated version of actor/actress first/last name
actors_to_get['format'] = actors_to_get.Actor.apply(
    lambda x: x.replace('-', '').replace('.', '').replace(" ", '').lower())

In [ ]:
# Build links to scrape BOM actor/actress pages for movie titles
actors_to_get['bomLink'] = actors_to_get.Actor.apply(
    lambda x: "http://www.boxofficemojo.com/people/chart/?view=Actor&id={}.htm".format(
        x.replace('-', '').replace('.', '').replace(" ", '').lower()))

In [ ]:
# Create a list of actor/actress links for scraping
actors_link_list = actors_to_get['bomLink'].tolist()

In [ ]:
# Clean actor/actress link list of bad links (not availabe on BOM)
actors_drop = ['http://www.boxofficemojo.com/people/chart/?view=Actor&id=davidkoechner.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=diedrichbader.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=dwaynejohnson.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=tikasumpter.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=robbieamell.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=chlo\xc3\xabgracemoretz.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=victoriajustice.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=essenceatkins.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=maiarawalsh.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=jacksonnicoll.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=brittanysnow.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=jeffdaniels.htm',
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=laurieholden.htm']

actors_link_list_cl = []
for link in actors_link_list:
    if link not in actors_drop:
        actors_link_list_cl.append(link)

In [ ]:
# Scrape BOM for a list of titles for actor/actress movie history
movies_to_get = get_title_bomlinks_by_actor(actors_link_list_cl)

In [ ]:
# Scrape BOM using the list of titles by actor/actress to scrape BOM
# movie data for actor/actress film history
movies_actors = get_bom_title_data(movies_to_get)

In [ ]:
# Create a crosswalk between actor name and movie title for matching movie data back to actors
actor_title_crosswalk = get_title_actor_df(actors_link_list_cl)

In [ ]:
# Append actor/actress names to BOM movie data
actor_titles_bom = pd.merge(
    actor_title_crosswalk, movies_actors, on = 'Title').sort_values(by = 'ReleaseDate')

In [ ]:
# Clear out additional bad records. Actors with count of 282 titles are from missing 
# BOM links for actors/actresses
actor_good = actor_titles_bom[['Actors', 'Title']].groupby(
    'Actors', as_index = False).count()[actor_titles_bom[['Actors', 'Title']].groupby(
        'Actors', as_index = False).count()['Title']< 282]
actor_good.columns = ['Actors', 'FilmCnt']

In [ ]:
# Cut down the dataframe with actor names and BOM movie data to only good actor/actress records
actor_titles_bom_good = pd.merge(actor_titles_bom, actor_good, on = 'Actors').sort_values(
    by = 'ReleaseDate')

In [ ]:
actor_titles_bom_good['DtgMillions'] = actor_titles_bom_good['DomesticTotalGross']/1000000
actor_titles_bom_good['Budget'] = actor_titles_bom_good['Budget'].replace('N/A', np.nan)

In [ ]:
# Add cumulative Domestic Total Gross average score for each actor and movie combination
actor_titles_dtg_score = add_rolling_average(
    actor_titles_bom_good, 'ReleaseDate', 'DtgMillions', 'Actors', 'DtgRollingAvg').sort_values(
by = 'Title')

In [ ]:
actor_titles_dtg_score.to_csv('DTG_SCORE.csv', index = False)

In [122]:
# Save crosswalk to csv
actor_title_crosswalk.to_csv("actor_title_crosswalk.csv", index = False)

# Step 3
Append actor scores to the original list of comedy movies since 2000 and split dataset into a training and a test set

In [ ]:
df_score = pd.read_csv('DTG_Score.csv')
movie_omdb = pd.read_csv("movies_comedy_omdb.csv")

In [ ]:
# Take data at actor-film level and create columns for number of actors to collapse to 
# film level and retain the first 4 actors listed.
df_unstack = df_score[['Actors', 'Title', 'DtgRollingAvg']]
df_unstack['block'] = (df_unstack['Title'] != df_unstack['Title'].shift(1)).astype(int).cumsum()
df_unstack['ActorCnt'] = df_unstack.groupby('block').transform(lambda x: (range(1, len(x) + 1)))
df_pivot = df_unstack.pivot(index='Title', columns='ActorCnt', values='DtgRollingAvg')
df_pivot_4 = df_pivot[[1, 2, 3, 4]]
df_pivot_4.columns = ['Actor1', 'Actor2', 'Actor3', 'Actor4']
df_pivot_4 = df_pivot.replace(-1, np.nan)

In [ ]:
# Create multiple score options per film that combines the scores of the actors listed
df_pivot_4['Score_avg'] = df_pivot_4.mean(axis=1)
df_pivot_4['Score_max'] = df_pivot_4.max(axis=1)
df_pivot_4['Score_product'] = df_pivot_4.product(axis=1)
df_pivot_4['Title'] = df_pivot_4.index

In [ ]:
# Get unique BOM data by movie title and append film scores
df_titles = df_score.drop(['Actors', 'FilmCnt', 'DtgRollingAvg'], axis = 1)
df_titles.drop_duplicates('Title', inplace = True)
df_titles_scored = pd.merge(df_titles, df_pivot_4, on = 'Title')

In [ ]:
# Cut down original list of comedy film data to year 2000 or more where Actor data is present
movie_omdb_2000 = movie_omdb[(movie_omdb['Year'] >= 2000) & (pd.notnull(movie_omdb['Actors']))]

In [ ]:
# Create a common version of the movie title to merge the original comedy movie data from
# the OMDB API with the BOM movie data. Strip out all punctuation and whitespace.
movie_omdb_2000['TitleMatch'] = movie_omdb_2000.Title.apply(
    lambda x: x.replace(" ",'').replace("'",'').replace(",",'').replace(".", '').replace(
        "!",'').replace(":", '').replace("-",'').replace("?",'').replace("&", '').lower())
df_titles_scored['TitleMatch'] = df_titles_scored.Title.apply(
    lambda x: x.replace(" ",'').replace("'",'').replace(",",'').replace(".", '').replace(
        "!",'').replace(":", '').replace("-",'').replace("?",'').replace("&", '').lower())
movie_merge = pd.merge(movie_omdb_2000, df_titles_scored, on = 'TitleMatch')

In [ ]:
# Create additional variables for modeling
movie_merge['Budget'] = movie_merge.Budget.apply(lambda x: float(str(x).replace(",",'')))
movie_merge['BudgetMillions'] = movie_merge['Budget']/1000000.0
movie_merge['Log_Score_avg'] = np.log(movie_merge['Score_avg'])
movie_merge['Log_Score_max'] = np.log(movie_merge['Score_max'])
movie_merge['Log_Score_product'] = np.log(movie_merge['Score_product'])
movie_merge['Log_DtgMillions'] = np.log(movie_merge['DtgMillions'])

In [ ]:
# Split data into train and test datasets
msk = np.random.rand(len(movie_merge)) < 0.7
train = movie_merge[msk]
test = movie_merge[~msk]

In [ ]:
train.to_csv('Train.csv', index = False)

In [ ]:
test.to_csv('Test.csv', index = False)